# Retriving all RNA-seq experiments from SRA

I used pysradb to retrieve all RNA-seq experiments from SRA. I used the following command:

In [ ]:
! pysradb search "rna-seq Populus tremula X Populus alba" --saveto Populus_tremula_X_Populus_alba.tsv

# some data wrangling

some studies were poorly annotated and I had to know how many samples were in the table. I used the following command:

In [ ]:
import pandas as pd
sra_table = pd.read_csv("Populus_tremula_X_Populus_alba.tsv", sep="\t", index_col=0)
print(sra_table.shape)
tree_sra_table = sra_table[sra_table.organism_name == "Populus tremula x Populus alba"]
print(tree_sra_table.shape)
print(len(tree_sra_table.index.unique()))
tree_sra_table.value_counts("experiment_desc")
print(tree_sra_table.value_counts("study_accession")) # used this to find out some study accessions have only one sample

# script generation - download all samples

In [5]:
with open("sbatch.sh", 'w') as ff:
    for SRP in tree_sra_table.index.unique():
        ff.write("sbatch " + f"{SRP}.sh" + "\n")
        with open(f"{SRP}.sh", 'w') as f:
            f.write(f"""#!/bin/bash
#SBATCH --job-name=pysradb         # Job name
#SBATCH --partition=highmem_p             # Partition (queue) name
#SBATCH --ntasks=1            # Run on a single CPU
#SBATCH --cpus-per-task=8
#SBATCH --mem=100gb                     # Job memory request
#SBATCH --time=168:00:00               # Time limit hrs:min:sec
#SBATCH --output=sra.%j.out    # Standard output log
#SBATCH --error=sra.%j.err     # Standard error log
#SBATCH --mail-type=END,FAIL          # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=ch29576@uga.edu  # Where to send mail

date
cd $SLURM_SUBMIT_DIR
pysradb download -y -t 8 -p {SRP}""")

# script generation - from .sra to .fastq

In [98]:
with open("sbatch_sra2fastq.sh", 'w') as ff:
    for SRP in tree_sra_table.index.unique():
        ff.write("sbatch " + f"./script/{SRP}_sra2fastq.sh" + "\n")
        with open(f"./script/{SRP}_sra2fastq.sh", 'w') as f:
            f.write(f"""#!/bin/bash
#SBATCH --job-name=pysradb         # Job name
#SBATCH --partition=highmem_p             # Partition (queue) name
#SBATCH --ntasks=1            # Run on a single CPU
#SBATCH --cpus-per-task=8
#SBATCH --mem=100gb                     # Job memory request
#SBATCH --time=168:00:00               # Time limit hrs:min:sec
#SBATCH --output=sra.%j.out    # Standard output log
#SBATCH --error=sra.%j.err     # Standard error log
#SBATCH --mail-type=ALL          # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=ch29576@uga.edu  # Where to send mail

date
ml SRA-Toolkit
cd $SLURM_SUBMIT_DIR
""")
            current_df = tree_sra_table.loc[SRP]
            if type(current_df) != pd.core.series.Series:
                for SRP_,info in current_df.iterrows():
                    f.write(f"fasterq-dump --split-3 ./{SRP_}/{info.experiment_accession}/{info.run_accession}.sra -O ./{SRP_}/fastq\n")
                
            else:
                f.write(f"fasterq-dump --split-3 ./{SRP}/{current_df.experiment_accession}/{current_df.run_accession}.sra -O ./{SRP}/fastq\n")
            f.write(f"date\n")

# script generation - prepare files for nfcore/rnaseq pipeline

here requires some manual work. some samples are not paired-end and some samples were not converted correctly in the previous step. I have to edit the file manually for a few cases.

In [9]:
with open("arrange_nfcore_rnaseq.sh", 'w') as ff:
    for SRP in tree_sra_table.index.unique():

        # # gzip all fastq
        # ff.write(f"cd {SRP}/fastq\n")
        # ff.write(f"rm -rf run.sh\n")
        # ff.write(f"""echo '#!/bin/sh' > run.sh\n""")
        # ff.write(f"""echo '#SBATCH -J {SRP}_gzip' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --partition batch' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --ntasks=1' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --cpus-per-task=8' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --time=124:00:00' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --mem=16gb' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --mail-user=ch29576@uga.edu' >> run.sh\n""")
        # ff.write(f"""echo '#SBATCH --mail-type=ALL' >> run.sh\n""")
        # ff.write(f"""echo 'date' >> run.sh\n""")
        # ff.write(f"""echo 'cd $SLURM_SUBMIT_DIR' >> run.sh\n""")
        # ff.write(f"""echo 'gzip *.fastq' >> run.sh\n""")
        # ff.write(f"""echo 'date' >> run.sh\n""")
        # ff.write(f"sbatch run.sh\n")
        # ff.write(f"cd ../../\n")

        # create input.csv
        ff.write(f"cd {SRP}\n")
        ff.write(f"rm -rf input.csv\n")
        ff.write(f"echo 'sample,fastq_1,fastq_2,strandedness' > input.csv\n")
        current_df = tree_sra_table.loc[SRP]
        if type(current_df) != pd.core.series.Series:
            if current_df["library_layout"][0] == "SINGLE":
                for SRP_,info in current_df.iterrows():
                    ff.write(f"""echo '{info.run_accession},./fastq/{info.run_accession}.fastq.gz,,unstranded' >> input.csv\n""")
    
        #     else:
        #         for SRP_,info in current_df.iterrows():
        #             ff.write(f"""echo '{info.run_accession},./fastq/{info.run_accession}_1.fastq.gz,./fastq/{info.run_accession}_2.fastq.gz,unstranded' >> input.csv\n""")
        # else:
        #     ff.write(f"""echo '{current_df.run_accession},./fastq/{current_df.run_accession}_1.fastq.gz,./fastq/{current_df.run_accession}_2.fastq.gz,unstranded' >> input.csv\n""")
        ff.write(f"cd ..\n")

        # # create nf-params.json
        # ff.write(f"cd {SRP}\n")
        # ff.write(f"rm -rf nf-params.json\n")
        # ff.write(f"mkdir results\n")
        # # let the shell script creaete a file with the following content
        # # {
        # #     "input": ".\/input.csv",
        # #     "outdir": ".\/results",
        # #     "email": "ch29576@uga.edu",
        # #     "multiqc_title": "2022SUT4",
        # #     "fasta": "\/scratch\/ch29576\/v5_717\/reference\/717v5.fa",
        # #     "gff": "\/scratch\/ch29576\/v5_717\/reference\/717v5.gff",
        # #     "gtf_extra_attributes": "Name",
        # #     "featurecounts_group_type": "",
        # #     "featurecounts_feature_type": "",
        # #     "skip_biotype_qc": true,
        # #     "pseudo_aligner": "salmon",
        # #     "aligner": "star_salmon"
        # # }
        # ff.write(f"echo '{{' > nf-params.json\n")
        # ff.write(f"""echo '    "input": ".\/input.csv",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "outdir": ".\/results",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "email": "ch29576@uga.edu",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "multiqc_title": "{SRP}",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "fasta": "\/scratch\/ch29576\/v5_717\/reference\/717v5.fa",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "gff": "\/scratch\/ch29576\/v5_717\/reference\/717v5.gff",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "gtf_extra_attributes": "Name",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "featurecounts_group_type": "",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "featurecounts_feature_type": "",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "skip_biotype_qc": true,' >> nf-params.json\n""")
        # ff.write(f"""echo '    "pseudo_aligner": "salmon",' >> nf-params.json\n""")
        # ff.write(f"""echo '    "aligner": "star_salmon"' >> nf-params.json\n""")
        # ff.write(f"echo '}}' >> nf-params.json\n")
        # ff.write(f"cd ..\n")

        # create run.sh
        ff.write(f"cd {SRP}\n")
        ff.write(f"rm -rf run.sh\n")
        ff.write(f"""echo '#!/bin/sh' > run.sh\n""")
        ff.write(f"""echo '#SBATCH -J {SRP}_RNA-Seq' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --partition highmem_p' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --ntasks=1' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --cpus-per-task=28' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --time=168:00:00' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --mem=400gb' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --mail-user=ch29576@uga.edu' >> run.sh\n""")
        ff.write(f"""echo '#SBATCH --mail-type=ALL' >> run.sh\n""")
        ff.write(f"""echo 'date' >> run.sh\n""")
        ff.write(f"""echo 'cd $SLURM_SUBMIT_DIR' >> run.sh\n""")
        ff.write(f"""echo 'ml Nextflow' >> run.sh\n""")
        ff.write(f"""echo 'NXF_SINGULARITY_CACHEDIR=/scratch/ch29576/singularity_cache' >> run.sh\n""")
        ff.write(f"""echo 'nextflow run nf-core/rnaseq -r 3.8.1 -name {SRP}-SE --outdir results -profile singularity -params-file nf-params.json -resume ' >> run.sh\n""")
        ff.write(f"""echo 'date' >> run.sh\n""")
        if current_df["library_layout"][0] == "SINGLE":
            ff.write(f"sbatch run.sh\n")
        ff.write(f"cd ..\n")
        

# Metadata from bioPython

I tried to use pysradb to get metadata but did not get what I want. So here I use bioPython to get the study abstract, which provided a little bit more info fo reach study.

In [11]:
! pysradb metadata SRP364200 --detailed --saveto SRP364200_metadata.tsv

In [40]:
# read all SRP and query the study abstract & save to csv
import Bio
from Bio import Entrez
import bs4 as bs
import requests
import lxml
import pandas as pd
from tqdm import tqdm

sra_table = pd.read_csv("Populus_tremula_X_Populus_alba.tsv", sep="\t", index_col=0)
tree_sra_table = sra_table[sra_table.organism_name == "Populus tremula x Populus alba"]
tree_sra_table
SRP_list = list(tree_sra_table.index.unique())

attr_list = [
    'EXTERNAL_ID',
    'STUDY_TITLE',
    'STUDY_ABSTRACT',
    # 'SUBMITTER_ID',
    'Name',
]
srp_df = pd.DataFrame(columns=(attr_list + ["SRP"]))
srp_df.set_index("SRP", inplace=True)
Entrez.email = "jgi@gmail.com"

# query the study abstract
for SRP in tqdm(SRP_list):

    
    handle = Entrez.esearch(db="sra", term=SRP, retmax=100000)
    record = Entrez.read(handle)
    handle.close()
    for id in record["IdList"]:
        handle = Entrez.efetch(db="sra", id=id)
        record = handle.read()
        handle.close()
    file = bs.BeautifulSoup(record, features="xml")
    for attr in attr_list:
        try:
            srp_df.loc[SRP, attr] = file.find(attr).get_text()
        except:
            srp_df.loc[SRP, attr] = "NA"
    

srp_df

,EXTERNAL_ID,STUDY_TITLE,STUDY_ABSTRACT,Name
SRP,,,,
SRP364200,PRJNA631840,Populus tremula x Populus alba Transcriptome o...,Transcriptome in in Poplar in response to nitr...,Chinese Academy of Forestry
SRP350268,GSM5727288,Single nuclei transcriptomic of Populus tremul...,Here we applied a novel approach to isolate nu...,NCBI
SRP349172,PRJNA785328,717-1B4 XBAT35 Root RNA-Seq,Transgenic 717-1B4 XBAT35 RNA-Seq from 12d root,Oak Ridge National Laboratory
SRP343659,GSM5660932,Transcriptome profiling of glycosyltransferase...,Comparison of UGT71L1 knock-out genes with emp...,NCBI
SRP331923,PRJNA753499,RNAseq data from poplar leaf development,RNAseq data of poplar leaf from three developm...,University of Georgia
...,...,...,...,...
SRP059838,PRJNA275366,Populus: Defoliation experiment,Carbon partitioning in poplar plant.,University of Georgia
SRP042117,PRJNA247385,Xylem transcriptome response of transgenic Pop...,Wild type and transgenic Populus plants were s...,University of Georgia
SRP058772,PRJNA284622,Populus tremula x Populus alba cultivar:Populu...,Study of gene expression in different wood typ...,USDA FOREST SERVICE


In [43]:
srp_df.to_csv("Populus_tremula_X_Populus_alba_srp_abstract.csv")